In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import json
from collections import OrderedDict
import os

Define Path

In [2]:
path_openlabel = '/home/varunsingh/_Carla/SafetyPool-API/sp-api/Generated OpenLabel/2CAA3189-3A7C-4E20-8FE8-CDA821B66B23openlabel.sdl'


Read Data

In [3]:
# Read the OpelLabelDict file
with open(path_openlabel, 'r') as file:
    data_openlabel = file.read()
# Dictionary of open labels
dict_openlabel = json.loads(data_openlabel)


Parse OpenLabel Dict to create rows for dataframe

In [39]:
row_dic_template = OrderedDict()
row_dic_template = {"ScenarioID":"","ScenarioDescription":"","ParentReference":"","OwnerName":"","VersionInfo":"","SceneryElements":"","DynamicElements":"","EnvironmentElement":"","TrafficElements":"","HazardLevel":""}
# dict_openlabel["tags"].values()

In [5]:
# function to parse Scenario Definition
def parseScenarioDefinition(scenariostr,row_dic):
    '''
    This function seperates out the Version info, Scenery elements, Dynamic elements
    Traffic elements, and Environment elements 
    '''
    scenariodic = {}
    scenarioidx = []

    if  'VERSION:' in scenariostr:
        scenariodic["VersionInfo"] = scenariostr.index('VERSION:')
        scenarioidx.append(scenariostr.index('VERSION:'))

    if 'SCENERY ELEMENTS:' in scenariostr:
        scenariodic["SceneryElements"] = scenariostr.index('SCENERY ELEMENTS:')
        scenarioidx.append(scenariostr.index('SCENERY ELEMENTS:'))

    if 'DYNAMIC ELEMENTS:' in scenariostr:
        scenariodic["DynamicElements"] = scenariostr.index('DYNAMIC ELEMENTS:')
        scenarioidx.append(scenariostr.index('DYNAMIC ELEMENTS:'))

    if 'ENVIRONMENT ELEMENTS:' in scenariostr:
        scenariodic["EnvironmentElement"] = scenariostr.index('ENVIRONMENT ELEMENTS:')
        scenarioidx.append(scenariostr.index('ENVIRONMENT ELEMENTS:'))

    if 'TRAFFIC ELEMENTS:' in scenariostr:
        scenariodic["TrafficElements"] = scenariostr.index('TRAFFIC ELEMENTS:')
        scenarioidx.append(scenariostr.index('TRAFFIC ELEMENTS:'))
    

    scenarioidx.sort()
    scenarioidx.append(None)
    
    inverted_dict = {v: k for k, v in scenariodic.items()}
    for i in range(0,len(scenarioidx)-1):
        row_dic[inverted_dict[scenarioidx[i]]] = scenariostr[scenarioidx[i]:scenarioidx[i+1]]

In [6]:
# Function to parse the whole openlabel
def parseOpenLabelTags(dict_openlabel,row_dic):
    for k,v in dict_openlabel["tags"].items():
        # Hazard Level
        if 'hazard' in v["type"].lower():
            row_dic["HazardLevel"] = v["type"]

        # UniqueReference
        if 'scenariouniquereference' in v["type"].lower():
            row_dic["ScenarioID"] = v["tag_data"]["text"][0]["val"]

        # ScenarioParentReference
        if 'scenarioparentreference' in v["type"].lower():
            row_dic["ParentReference"] = v["tag_data"]["text"][0]["val"]

        # ScenarioDescription
        if 'scenariodescription' in v["type"].lower():
            row_dic["ScenarioDescription"] = v["tag_data"]["text"][0]["val"]

        # OwnerName
        if 'ownername' in v["type"].lower():
            row_dic["OwnerName"] = v["tag_data"]["text"][0]["val"]

        # scenarioDefinition
        if 'scenariodefinition' in v["type"].lower():
            scenariostr = v["tag_data"]["text"][0]["val"]
            parseScenarioDefinition(scenariostr,row_dic)
        
    return row_dic

Test

In [27]:
r1 = parseOpenLabelTags(dict_openlabel,row_dic_template.copy())


In [28]:
r1

{'ScenarioID': '2caa3189-3a7c-4e20-8fe8-cda821b66b23',
 'ScenarioDescription': 'ALKS Follow Lead Vehicle Emergency Brake',
 'ParentReference': 'a5327c50-203e-4e59-822c-78d6a0e0617f',
 'OwnerName': 'WMG',
 'VersionInfo': "VERSION: 8.1\nEXTENSION: None\nAUTHOR: 'WMG, Intelligent Vehicles - V&V Team (https://warwick.ac.uk/fac/sci/wmg/research/cav/vandv/)'\n\n",
 'SceneryElements': 'SCENERY ELEMENTS:\nDO: Map - roads and junctions network [Network1] as: \nJunctions: None\nRoads:\nR1: \nSTART\nRoad type [Motorway] as [R1] with zone as [N/A] AND speed limit of [N/A] in an [N/A] environment with \n Number of lanes [3] as [R1.L1, R1.L2, R1.L3] \n Road traffic direction [Right-handed] \n Lane type [Traffic lane]   \n Lane markings [Broken line] \n Horizontal road geometry [Straight] \n Vertical road geometry [Level plane]\n Transverse road geometry [Divided] \n Length [9000 to 11000] AND Lane width [3.4 to 3.7] \nEND\n\n',
 'DynamicElements': 'DYNAMIC ELEMENTS:\nINITIAL: \tVehicle [Ego] in [R1.

Create a DF

In [9]:
import pandas as pd

In [29]:
# df = pd.DataFrame([r1])
df = pd.DataFrame([row_dic_template])

In [38]:

row_dic_template

{'ScenarioID': '2caa3189-3a7c-4e20-8fe8-cda821b66b23',
 'ScenarioDescription': 'ALKS Follow Lead Vehicle Emergency Brake',
 'ParentReference': 'a5327c50-203e-4e59-822c-78d6a0e0617f',
 'OwnerName': 'WMG',
 'VersionInfo': "VERSION: 8.1\nEXTENSION: None\nAUTHOR: 'WMG, Intelligent Vehicles - V&V Team (https://warwick.ac.uk/fac/sci/wmg/research/cav/vandv/)'\n\n",
 'SceneryElements': 'SCENERY ELEMENTS:\nDO: Map - roads and junctions network [Network1] as: \nJunctions: None\nRoads:\nR1: \nSTART\nRoad type [Motorway] as [R1] with zone as [N/A] AND speed limit of [N/A] in an [N/A] environment with \n Number of lanes [3] as [R1.L1, R1.L2, R1.L3] \n Road traffic direction [Right-handed] \n Lane type [Traffic lane]   \n Lane markings [Broken line] \n Horizontal road geometry [Straight] \n Vertical road geometry [Level plane]\n Transverse road geometry [Divided] \n Length [9000 to 11000] AND Lane width [3.4 to 3.7] \nEND\n\n',
 'DynamicElements': 'DYNAMIC ELEMENTS:\nINITIAL: \tVehicle [Ego] in [R1.

In [19]:
df.to_excel('example.xlsx', index=False)

Do for multiple files 

In [70]:
folder_path = '/home/varunsingh/_Carla/SafetyPool-API/sp-api/Generated OpenLabel'

files = os.listdir(folder_path)
df = pd.DataFrame([])
records = []
for file in files:

    file_path = os.path.join(folder_path,file)
    with open(file_path,'r') as file:
        content  = file.read()
        dict_openlabel = json.loads(content)
        row = parseOpenLabelTags(dict_openlabel.copy(),row_dic_template.copy())
        # print(row)
        records.append(row)
    
    

df = pd.DataFrame.from_records(records)
df
df.to_excel('example.xlsx', index=False)
        



,ScenarioID,ScenarioDescription,ParentReference,OwnerName,VersionInfo,SceneryElements,DynamicElements,EnvironmentElement,TrafficElements,HazardLevel
0,39cc450e-79f7-4f2f-8644-d1ab07ba60cb,stat19_2_1027_p,90627ca0-9886-4fa6-ba9b-213b9f25bcab,WMG,"VERSION: 8.1\nEXTENSION: None\nAUTHOR: 'WMG, I...",SCENERY ELEMENTS:\nDO: Map - roads and junctio...,DYNAMIC ELEMENTS:\nINITIAL: Vehicle [Ego] in [...,ENVIRONMENT ELEMENTS:\nDO: [Env1]\nWind [0 to ...,,HazardMinor
1,fa600436-df84-49d1-90d2-04d73184a22c,ALKS Fully Blocking Target,9cd3ffa4-7bfe-438a-b28c-0f6b3b4c3e61,WMG,"VERSION: 8.1\nEXTENSION: None\nAUTHOR: 'WMG, I...",SCENERY ELEMENTS:\nDO: Map - roads and junctio...,DYNAMIC ELEMENTS:\nINITIAL: \tVehicle [Ego] in...,ENVIRONMENT ELEMENTS:\nDO: [Env1]\n Wind [0 t...,,
2,915aea8f-e173-47d4-a98c-4a1fff28c21f,ALKS Lateral Detection Range,930bff3e-2171-4617-a191-4a0f59722d3c,WMG,"VERSION: 8.1\nEXTENSION: None\nAUTHOR: 'WMG, I...",SCENERY ELEMENTS:\nDO: Map - roads and junctio...,DYNAMIC ELEMENTS:\nINITIAL: \tVehicle [Ego] in...,ENVIRONMENT ELEMENTS:\nDO: [Env1]\n Wind [0 t...,,
3,9b0cb37d-924c-41a9-ade5-59b378e695c7,ALKS Cut-Out Fully Blocking,b88b450c-4072-4ead-a93e-fd51bbda4ec3,WMG,"VERSION: 8.1\nEXTENSION: None\nAUTHOR: 'WMG, I...",SCENERY ELEMENTS:\nDO: Map - roads and junctio...,DYNAMIC ELEMENTS:\nINITIAL: \tVehicle [Ego] in...,ENVIRONMENT ELEMENTS:\nDO: [Env1]\n Wind [0 t...,,
4,2caa3189-3a7c-4e20-8fe8-cda821b66b23,ALKS Follow Lead Vehicle Emergency Brake,a5327c50-203e-4e59-822c-78d6a0e0617f,WMG,"VERSION: 8.1\nEXTENSION: None\nAUTHOR: 'WMG, I...",SCENERY ELEMENTS:\nDO: Map - roads and junctio...,DYNAMIC ELEMENTS:\nINITIAL: \tVehicle [Ego] in...,ENVIRONMENT ELEMENTS:\nDO: [Env1]\n Wind [0 t...,,
5,15c613e8-3535-4e0f-8a88-8c12a45f6248,ALKS Swerving Lead Vehicle,ec9eab33-3544-4ef3-8789-3271e1395205,WMG,"VERSION: 8.1\nEXTENSION: None\nAUTHOR: 'WMG, I...",SCENERY ELEMENTS:\nDO: Map - roads and junctio...,DYNAMIC ELEMENTS:\nINITIAL: \tVehicle [Ego] in...,ENVIRONMENT ELEMENTS:\nDO: [Env1]\n Wind [0 t...,,
6,f382f977-fe86-48ef-8886-cec80628d806,claim_1_12785,8e4a1b1d-a98d-41cc-a728-bb0fd3c8a3ba,WMG,"VERSION: 8.1\nEXTENSION: None\nAUTHOR: 'WMG, I...",SCENERY ELEMENTS:\nDO: Map - roads and junctio...,DYNAMIC ELEMENTS:\nINITIAL: Vehicle [Ego] in [...,ENVIRONMENT ELEMENTS:\nDO: [Env1]\nWind [10.8 ...,,HazardMajor
7,f13189b3-8330-4fc8-9192-3a7cab3e14a4,claim_1_10915,556199ac-ac81-48d1-8cfc-7a833048c00d,WMG,"VERSION: 8.1\nEXTENSION: None\nAUTHOR: 'WMG, I...",SCENERY ELEMENTS:\nDO: Map - roads and junctio...,DYNAMIC ELEMENTS:\nINITIAL: Vehicle [Ego] in [...,ENVIRONMENT ELEMENTS:\nDO: [Env1]\nWind [20.8 ...,,HazardMajor
8,e6da1894-afaa-4644-a8ad-631d4773369d,UCA_3.1.2_TS_2,cb48bc4d-e47d-41b5-a4ed-ba35065d1474,WMG,"VERSION: 8.1\nEXTENSION: None\nAUTHOR: 'WMG, I...",SCENERY ELEMENTS:\nDO: Map - roads and junctio...,DYNAMIC ELEMENTS:\nINITIAL: Vehicle [Ego] in [...,ENVIRONMENT ELEMENTS:\nDO: [Env1]\nWind [0 to ...,,
9,ef43abe0-add9-4d12-9cbd-3329ebe2949e,ALKS Partially Blocking Target,4d15dfdf-c15b-4a20-aa66-ec3a22db2982,WMG,"VERSION: 8.1\nEXTENSION: None\nAUTHOR: 'WMG, I...",SCENERY ELEMENTS:\nDO: Map - roads and junctio...,DYNAMIC ELEMENTS:\nINITIAL: \tVehicle [Ego] in...,ENVIRONMENT ELEMENTS:\nDO: [Env1]\n Wind [0 t...,,
